# 統計解析(2)
Scipyを用いた各種統計検定・推定、Scikit-Learnを用いた機械学習について触れ、ゲノム解析にどの様に活用出来るのかを学ぶ。

## Contents

### 機械学習と統計学の違い、ゲノム解析への応用について
1. [スライド](#0.1)

参考:[Machine Learning vs. Statistics](https://www.svds.com/machine-learning-vs-statistics)

### 前回の復習・補足
1. [重回帰分析](#1.1)
1. [問題点](#1.2)
1. [解決法](#1.3)

### 今回の実習
1. [機械学習の基本的な流れ](#2.1)
1. [変数選択手法の一例](#2.2)
1. [決定木](#2.3)
1. [データの準備](#2.4)

## 前回の復習・補足

### 1. 重回帰分析<a name="1.1"></a>

説明変数によって目的変数の値を説明しようとするモデル。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2018/09_statistics/data/regression_base.png?raw=true" alt="reg_base" width="50%" height="50%">

\begin{align}
\boldsymbol{y} = \boldsymbol{X} \boldsymbol{\beta} + \boldsymbol{e}
\end{align}

\begin{align}
\boldsymbol{y} = \left[\begin{array}{c}
            y_1 \\
            y_2 \\
            ... \\
            y_n \\
        \end{array}\right] \quad
\boldsymbol{X} = \left[\begin{array}{c}
            x_{11} & x_{21} & x_{31} & ... & x_{k1} \\
            x_{12} & x_{22} & x_{32} & ... & x_{k2} \\
            ... & ... & ... & ... & ...\\
            x_{1n} & x_{2n} & x_{3n} & ... & x_{kn} \\
        \end{array}\right] \quad
\boldsymbol{\beta} = \left[\begin{array}{c}
            \beta_1 \\
            \beta_2 \\
            ... \\
            \beta_k \\
        \end{array}\right] \quad
\boldsymbol{e} = \left[\begin{array}{c}
            e_1 \\
            e_2 \\
            ... \\
            e_n \\
        \end{array}\right]
\end{align}

$\hat{y_i} = \boldsymbol{X_i} \boldsymbol{\beta}$, $\sum{e_i^2} = \sum{(\hat{y_i} - y_i)^2}$を最小とする$\boldsymbol{\beta}$を$\boldsymbol{X'}\boldsymbol{X}\boldsymbol{\beta} = \boldsymbol{X'}\boldsymbol{y}$を計算することによって求める。<br><br>
→ $\boldsymbol{\hat{\beta}} = (\boldsymbol{X'}\boldsymbol{X})^{-1}\boldsymbol{X'}\boldsymbol{y}$

In [ ]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model

# データの読み込み&変換
gene_data = pd.read_csv("https://github.com/CropEvol/lecture/blob/master/textbook_2018/09_statistics/data/gene_data.csv?raw=true", index_col=0)
convert_gene_data = pd.get_dummies(gene_data, drop_first=True)
convert_gene_data.head()

In [ ]:
# モデル選択
clf = linear_model.LinearRegression()

# 説明変数にchr10の塩基データ(４列目以降)を利用
X = convert_gene_data.iloc[:, 3:]

# 目的変数に "LeafWidth" を利用
Y = convert_gene_data.loc[:, "LeafWidth"]

# 予測モデルを作成
clf.fit(X, Y)

# 回帰係数
print(clf.coef_)
 
# 切片 (誤差)
print(clf.intercept_)
 
# 決定係数
print(clf.score(X, Y))

In [ ]:
# 各遺伝子の効果を見てみる
plt.figure(figsize=(25, 5), dpi=50)
col_names = convert_gene_data.iloc[:, 3:].columns.values
col_num = len(col_names)
plt.scatter(range(col_num), clf.coef_)
plt.xticks(range(col_num), col_names)
plt.show()

# 3,4,5番目の塩基の効果だけ特異的

### 2. 問題点<a name="1.2"></a>

#### 根本的な問題点
* 実際に効果のある塩基はchr3,4,5のものだけ　→　しかし、効果の無いはずの塩基の情報を加えると決定係数が上昇する。
* つまり、実際には意味の無い遺伝子情報でも、加えれば加えるほど決定係数が上がることになってしまう。


#### 生物学的な問題点
* 全ての塩基に効果があるという状況は不自然
* 遺伝子の数は更に多く、全てを考慮しようとすると上述した問題が発生。
* 遺伝子間相互作用など、変数間の交互作用の影響を考慮していない。
* ...etc

→　予測モデルを作るためのサンプルに関してのみ上手く説明できるが、新たなデータに対し当てはまりの悪い予測モデルになってしまう。<br>
　(Overfittingと呼ぶ)<br>

#### 数学的な問題点
* 遺伝子の効果を推定は$\boldsymbol{\hat{\beta}} = (\boldsymbol{X'}\boldsymbol{X})^{-1}\boldsymbol{X'}\boldsymbol{y}$により求める→ $\boldsymbol{X'}\boldsymbol{X}$の逆行列が存在する必要がある。
* あまりにも変数(遺伝子の数)が多いと全て0の列や相同な列が増え、行列のランク落ちという現象が生じ、逆行列が正しく求められない場合がある。
* ゲノムデータを線形回帰すると、連鎖の影響で上述した現象が生じやすい。
* この様な問題には"正則化"と呼ばれる手法を用いたりする。($\boldsymbol{\hat{\beta}} = (\boldsymbol{X'}\boldsymbol{X}+\lambda\boldsymbol{I_{p+1}})^{-1}\boldsymbol{X'}\boldsymbol{y}$の様にする)

In [ ]:
##### 無意味なデータをたくさん加えて回帰分析すると決定係数が上昇する例
import random

# yの値だけコピー
test_data = convert_gene_data.iloc[:, 0:2]

# 説明変数にランダムな塩基のデータを発生させる
for i in range(20):
    test_data["gene_test_{}".format(i)] = random.choices(["A", "T", "G", "C"], k=200)

test_data
# カテゴリカルデータの変換
convert_test_data = pd.get_dummies(test_data, drop_first=True)

# モデル選択
clf = linear_model.LinearRegression()
    
X = convert_test_data.iloc[:, 2:]
Y = convert_test_data.iloc[:, 1]

# 予測モデルを作成
clf.fit(X, Y)

# 決定係数
print(clf.score(X, Y))

# 各塩基の効果を見てみる
plt.title("the effect of each base")
plt.scatter(range(len(clf.coef_)), clf.coef_)
plt.show()

In [ ]:
##### 相同な列が存在すると、妥当な推定値が得られない例
import random

# 全く同じ遺伝子のデータを付け加える
test_data = convert_gene_data.iloc[:, 3:]
test_data = pd.concat([convert_gene_data, test_data], axis=1)

# モデル選択
clf = linear_model.LinearRegression()
    
X = test_data.iloc[:, 3:]
Y = test_data.iloc[:, 1]

# 予測モデルを作成
clf.fit(X, Y)

# 偏回帰係数
print(clf.coef_)

# 決定係数
print(clf.score(X, Y))

# 各遺伝子の効果を見てみる
plt.title("the effect of each gene")
plt.scatter(range(len(clf.coef_)), clf.coef_)
plt.show()

### 3. 解決策の例<a name="1.3"></a>

#### データを増やす
* 当たり前だが200個体ではなく1,000,000個体のデータから回帰式を作ることができれば…

#### 変数選択
出来る限り形質($y$)に関与していると考えられる説明変数のみで回帰式を作成することで、意味のある回帰式になると考えられる。

* 生物学的な選択方法
    * GWAS(Genome Wide Association Study)と呼ばれる手法で有意なものを選ぶ。

    
* 分析手法による選択方法
    * 決定係数とは異なった、モデルの妥当性の指標が存在する。
    * 変数選択や次元削減を伴った分析手法が存在する。


* ...etc

予測である以上完璧な方法というものは存在しない。求めたいもの(予測精度なのか、モデルの妥当性なのか、等)に応じて取捨選択をする。

#### では、そもそも意味のある回帰式・意味のあるモデルとは何をもって決めれば良いのだろうか？

---

## 今回の実習

### 機械学習の基本的な流れについて<a name="2.1"></a>

まず、作り出した回帰式・モデルがどのくらい現実的なもの(意味のあるもの)なのか、測定するにはどうすれば良いか。<br>
##### これまでの回帰分析
<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2018/10_statistics/data/data_split1.png?raw=true" alt="reg_base" width="60%" height="60%">
<br>
→新たなデータセットに対しても説明力があるか知りたい。<br>

→データを分割し、新たなデータセットとして扱うものを意図的に用意しておく<br>

##### 予測精度を計測するには(回帰分析の例)
<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2018/10_statistics/data/data_split2.png?raw=true" alt="reg_base" width="75%" height="75%">

In [ ]:
# データ分割の処理はtrain_test_splitという関数で行える
from sklearn.model_selection import train_test_split

# 説明変数にchr10の塩基データ(４列目以降)を利用
X = convert_gene_data.iloc[:, 3:]

# 目的変数に "LeafWidth" を利用
Y = convert_gene_data.loc[:, "LeafWidth"]

# Training DataとTest Dataに分ける
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0, test_size=0.1)

# 各Dataの数を確認
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# モデル選択
clf = linear_model.LinearRegression()

# Training Dataから予測モデルを作成
clf.fit(X_train, y_train)

# 回帰係数
print(clf.coef_)

# 切片 (誤差)
print(clf.intercept_)

# 決定係数
print(clf.score(X, Y))

# Test Dataでの決定係数
print(clf.score(X_test, y_test))

In [ ]:
# Test Dataにおける実測値と予測値の比較
predict_value = clf.predict(X_test)

plt.plot(range(len(y_test)), y_test, color="b")
plt.plot(range(len(y_test)), predict_value, color="r")

plt.title("Measured value vs Prediction")
plt.xlabel("n")
plt.ylabel("Sepal Width")
plt.show()

### 2. 変数選択手法の一例<a name="2.2"></a>

特にゲノムデータを解析する際には、前回の重回帰分析の例や、前回課題のExercise2の様に、実際には一部の変数しか効果を持たない場合も多い。<br>

事前にどの変数が効果を持っているのか分かっていれば良いが、その様な状況は殆ど無く、なんらかの基準で変数選択をする必要があるかもしれない。<br>

そこで、ここでは変数選択の手法を幾つか紹介する。

#### 2.1. 各変数毎の有意性検定

$y=\beta_1x_1+\beta_2x_2+\beta_3x_3...+e$<br>

この時、$H_0:\beta_1=0 \quad H_1:\beta_1\neq0$のような形で、各変数が効果を持つかどうか調べ、有意なものを選ぶ。<br>

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2018/10_statistics/data/yuui.png?raw=true" alt="reg_base" width="50%" height="50%">

`for`文で各変数の検定を行っても良いが、`statsmodels`という統計モデリング用のライブラリを使うと便利。<br>

※今回は統計モデリングについては詳しく触れないため、簡単に使ってみるのみに留める。<br>

In [ ]:
import statsmodels.api as sm

# 説明変数の設定
X = convert_gene_data.iloc[:, 3:]

# 目的変数の設定
Y = convert_gene_data.loc[:, "LeafWidth"]

# 誤差項の追加
X = sm.add_constant(X)
results = sm.OLS(endog=Y, exog=X).fit()
results.summary()

chr_3,chr_4,chr_5の塩基のp値が低く、有意であると考えられる。

また、統計モデリングに関しては、[データ解析のための統計モデリング入門](http://goo.gl/Ufq2)という本が評判は良い。
___

#### 2.2 AIC, BICを用いてモデル選択を行う

モデルの良さを測る指標として、<br>
* AIC(赤池情報量基準):予測能力が最良のモデルを良いとする指標<br>
* BIC(ベイズ情報量規準):真のモデルである確率が最も大きいモデルを良いとする指標<br>

などを用いる場合がある。幾つかモデルを作成し、AIC等を基準に良いモデルを選択する。

例)<br>
　$y = {gene_1}\_effect + {gene_2}\_effect + {gene_3}\_effect + e$...モデル(1), AIC=243.8<br>
　$y = {gene_1}\_effect + {gene_2}\_effect + e$...モデル(2), AIC=198.3<br>

　AICの小さいモデル(2)が予測能力が最良のモデルである。

AICやBICの計算にも、`statsmodels`が使える。

In [ ]:
# 全塩基を用いて作成したモデルのAIC, BIC
X = convert_gene_data.iloc[:, 3:]
Y = convert_gene_data.loc[:, "LeafWidth"]
X = sm.add_constant(X)
results = sm.OLS(endog=Y, exog=X).fit()
print("全塩基の情報を入れたモデルのAIC:", results.aic)
print("全塩基の情報を入れたモデルのBIC:", results.bic)

# 3,4,5番染色体の塩基のみを用いて作成したモデルのAIC, BIC
X = convert_gene_data.iloc[:, 5:14]
Y = convert_gene_data.loc[:, "LeafWidth"]
X = sm.add_constant(X)
results = sm.OLS(endog=Y, exog=X).fit()
print("3,4,5番染色体のみの情報を入れたモデルのAIC:", results.aic)
print("3,4,5番染色体のみの情報を入れたモデルのBIC:", results.bic)

##### 3,4,5番染色体のみの情報を入れたモデルの方が、AIC(BIC)が小さい。よってモデルとしては後者の方が良いと判断される。

___
#### 2.3 変数選択を伴う手法

変数選択を行う手法の一つであるLassoを紹介する。<br>

Lasso(Least absolute shrinkage and selection operator)[[Tibshirani, 1996]](http://statweb.stanford.edu/~tibs/lasso/lasso.pdf)<br>

2.1, 2.2の手法ではモデルの推定と変数選択は別々に行っていた。<br>
Lassoでは$\beta$の一部が0として推定され、モデルの推定と変数選択を同時に行うことが可能。



In [ ]:
# モデル選択(ここをLinearRegression()からLasso()に変えるだけ)
# alphaの値が大きくなるほど、選択する変数の数が少なくなる。
clf = linear_model.Lasso(alpha=0.01)

# Training Dataから予測モデルを作成
clf.fit(X_train, y_train)

# 回帰係数
print(clf.coef_)

# 切片 (誤差)
print(clf.intercept_)

# 決定係数
print(clf.score(X_train, y_train))

# Test Dataでの決定係数
print(clf.score(X_test, y_test))

##### (参考) L1正則化

* 重回帰分析の最小二乗法の場合<br>

　　$\boldsymbol{y} = \boldsymbol{X} \boldsymbol{\beta} + \boldsymbol{e}$<br>

　　$\sum{e_i^2} = \sum{(\boldsymbol{X_i} \boldsymbol{\beta} - y_i)^2}$を最小とする$\boldsymbol{\beta}$を求める。→　 $\boldsymbol{\hat{\beta}} = (\boldsymbol{X'}\boldsymbol{X})^{-1}\boldsymbol{X'}\boldsymbol{y}$
<br><br>

* Lasso(L1正則化)の場合

　　$\sum{(\boldsymbol{X_i} \boldsymbol{\beta} - y_i)^2} + \lambda||\boldsymbol{\beta}||_1 \quad (||\boldsymbol{\beta}||_1=\sum{|\beta_i|})$<br><br>
　　を最小とする$\boldsymbol{\beta}$を求める。　→　$||\boldsymbol{\beta}||_1$を微分出来ず、解析的には解けないため、数値計算的な手法により求める。<br><br>
　　[Coordinate Descent](https://core.ac.uk/download/pdf/6287975.pdf?repositoryId=153), [LARS](http://statweb.stanford.edu/~imj/WEBLIST/2004/LarsAnnStat04.pdf)といったアルゴリズムによる求め方が有名。

In [ ]:
# grouplassoとかの例
# なぜプログラムするのか

### 3. 決定木<a name="2.3"></a>

機械学習の手法の中でも、中身の理解しやすい手法として、決定木というものがある。<br>

データの特徴を説明しやすい手法であるので、ゲノム解析に限らず、実験系のデータを解釈する際にも便利かと思われる。<br>

また、機械学習の最新の手法の幾つかは、この決定木をベースに作られていることもあるため、知っておくと良い。<br>

##### 決定木の一例

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2018/10_statistics/data/tree.png?raw=true" alt="tree" width="50%" height="50%">

決定木とは上の様に、データを分割し、より分かり易い形に変えていく手法になる。

In [ ]:
# グラフ描写用
!conda install -y graphviz

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeRegressor(max_depth=3)

clf = clf.fit(X_train, y_train)

predicted = clf.predict(X_train)

print("Training DataのScore", clf.score(X_train, y_train))
print("Test DataのScore", clf.score(X_test, y_test))

# 決定木を描写するコード(無理矢理表示させているので、参考にはしない様に)
from sklearn.externals.six import StringIO
from IPython.display import Image
dot_data = StringIO()
with open("tree.dot", 'w') as f:
    tree.export_graphviz(clf, out_file=f, feature_names=convert_gene_data.iloc[:, 3:].columns.values, filled=True, rounded=True, impurity=False, proportion=False,)
!dot -T png tree.dot > tree.png
Image('tree.png')

決定木の弱点として、深い木になるほど過学習しやすいという点が挙げられる。<br>
max_depthの値を大きくするほど、Training Dataのスコアが上がり、Test Dataのスコアが下がっていく。(新たなデータセットに対する予測精度が下がる。)<br>

### 4. データの準備<a name="2.4"></a>

機械学習にはその他様々な手法がある。　(参考)[scikit-learnのアルゴリズムチートシート]("https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html")<br>

どれも便利で強力な手法だが、一方でGarbage in, Garbage outと呼ばれる概念がある様にデータの質というものもまた重要となる。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2018/10_statistics/data/gigo.png?raw=true" alt="gigo" width="50%" height="50%">

つまり、どれほど強力で完璧なモデルを利用しても、入力するデータの質が悪ければ、得られる予測結果などの質も悪くなる場合が多い。<br>

そういった意味で、データ解析を進める前に、解析するデータそのものの質というものを考える必要性は高い。<br>

また、農学・生物学の世界では、データを上手くコントロールすることで、より解釈のしやすい解析結果を得ることも可能になる。<br>

##### (例) RIL集団

例えば、RIL(Recombinant Inbred Lines)と呼ばれるサンプル集団がある。<br>

RILは、自家交配を繰り返すことで、ゲノムの構造をどちらかの親のホモ接合体のみに揃えた集団になる。<br>

この集団は、解析の際にヘテロな組み合わせを考慮する必要がなく、ゲノム解析を行う際には非常に便利なデータセットと言える。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2018/10_statistics/data/ril.png?raw=true" alt="gigo" width="50%" height="50%">

人を対象としない農学の分野では、対象となる作物や生物に関して、生育する環境や遺伝子構造をある程度コントロール出来る。<br>

そのため、どの様なデータセットをどのくらいの規模で作成するのか、といった点への意識や工夫が非常に重要である。